In [1]:
from spmf import Spmf
import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt

pd.options.display.max_colwidth = 500000

First the data is read into the program

In [3]:
data=pd.read_csv("data_sessions_20s_min_length_1.csv")

Now the number of sessions which each of the apps occure in is calculated.

In [4]:
app_in_n_sessions=data.groupby(['app_code','user_id']).session.nunique().reset_index().groupby("app_code").session.sum()

In [5]:
data=pd.merge(data, app_in_n_sessions.reset_index().rename(columns={'session':'number_of_sessions'}), how='left', on=['app_code'])

In [6]:
data['app_infrequent']=data.number_of_sessions<=(data.shape[0]/100*0.001)

In [7]:
data.shape[0]

181323205

In [8]:
data['new_session']=(data.new_session | data.app_infrequent)

In [9]:
data['updated_session']=data.groupby(['user_id']).agg({'new_session':'cumsum'}).astype('int')

In [11]:
data=data[data.app_infrequent!=True]

In [12]:
data=pd.merge(data,(data.groupby(['user_id','updated_session']).agg({'app_code':'size'})).reset_index().rename(columns={'app_code':'session_length_updated'}),how='left',on=['user_id','updated_session'])

In [13]:
data=data.drop(['session_length'],axis=1)

Create text file for algorithm input

In [14]:
input_string=data.groupby(['user_id','updated_session'])['app_code'].apply(list).apply(lambda l: ' -1 '.join([str(i) for i in l])+" -1 -2")

In [15]:
#update file name
input_string.to_csv("input_string_without_0_001_apps_internal_removed.csv",index=False)

In [2]:
#update file name
input_string=pd.read_csv("input_string_without_0_001_apps_internal_removed.csv")

In [ ]:
dfAsString = input_string.to_string(header=False, index=False).lstrip()

In [ ]:
dfAsString=re.sub(r'\n\s+','\n',dfAsString)

In [ ]:
#update file name
f = open(os.path.join(os.getcwd(), '20sec_sessions_without_0_001_apps_internal_removed.txt'), 'w')
f.write(dfAsString)
f.close()